In [2]:
pip install ace_tools

Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import numpy as np
import os

# 1) 기본 설정 ----------------------------------------------------------------
BASE = '.'
tsr_files = [
    'TSR_2024_코스피_처치추가_After포함.xlsx',
    'TSR_2024_코스닥_처치추가_After포함.xlsx',
    'TSR_2023_코스피_처치추가_After포함.xlsx',
    'TSR_2023_코스닥_처치추가_After포함.xlsx'
]
manufacture_file = '제조업통합.xlsx'
flag_col = '밸류업 참여 여부'          # 처치 여부 컬럼명

# 2) TSR 파일 결합 -------------------------------------------------------------
tsr = pd.concat([pd.read_excel(os.path.join(BASE, f)) for f in tsr_files],
                ignore_index=True)

tsr['회사명'] = tsr['회사명'].astype(str).str.strip()
tsr[flag_col] = tsr[flag_col].map(
    lambda x: 1 if str(x).strip().upper() in ['1', 'Y', 'YES', 'T', 'TRUE'] else 0
)

# 3) 제조업 기업명으로 필터 -----------------------------------------------------
mf_name = (pd.read_excel(os.path.join(BASE, manufacture_file))['회사명']
             .astype(str).str.strip().unique())
tsr = tsr[tsr['회사명'].isin(mf_name)].copy()

# 4) 연도 정규화 + 그룹 라벨 ----------------------------------------------------
def to_int_year(x):
    try:
        return int(str(int(float(x)))[:4])
    except Exception:
        return None

tsr['연도'] = tsr['회계년도'].apply(to_int_year)

def label(row):
    if row[flag_col] == 1 and row['연도'] == 2023: return 'Treatment_Pre'
    if row[flag_col] == 1 and row['연도'] == 2024: return 'Treatment_Post'
    if row[flag_col] == 0 and row['연도'] == 2023: return 'Control_Pre'
    if row[flag_col] == 0 and row['연도'] == 2024: return 'Control_Post'
    return 'Other'
tsr['Group'] = tsr.apply(label, axis=1)

# 5) 숫자 변환 + inf → NaN ------------------------------------------------------
target_cols = ['TSR', 'PBR', '[공통]자기자본순이익률(IFRS)']
tsr[target_cols] = (tsr[target_cols]
                    .apply(pd.to_numeric, errors='coerce')
                    .replace([np.inf, -np.inf], np.nan))

# 6) 그룹별 평균 계산 -----------------------------------------------------------
stats = (tsr.groupby('Group')[target_cols]
           .mean()                                   # NaN 자동 무시
           .rename(columns={'TSR': '평균 TSR',
                            'PBR': '평균 PBR',
                            '[공통]자기자본순이익률(IFRS)': '평균 ROE'})
        ).loc[lambda df: df.index != 'Other']        # 'Other' 제거

# 7) 결과 출력·저장 -------------------------------------------------------------
print('\n=== 4그룹 × 3지표 평균값 (Inf 제거) ===')
print(stats)

out_path = os.path.join(BASE, 'Group_Mean_Stats.csv')
stats.to_csv(out_path, encoding='utf-8-sig')
print(f'\n→ 결과가 {out_path} 로 저장되었습니다.')



=== 4그룹 × 3지표 평균값 (Inf 제거) ===
                   평균 TSR    평균 PBR    평균 ROE
Group                                        
Control_Post    -7.673024  2.710367 -4.478300
Control_Pre     16.789778  2.489595 -4.626897
Treatment_Post   1.806231  3.125933  6.183867
Treatment_Pre   32.635261  2.580667  7.652533

→ 결과가 .\Group_Mean_Stats.csv 로 저장되었습니다.


In [1]:
import pandas as pd
import numpy as np
import os

# ----------------------- 기본 설정 -----------------------
BASE = '.'
tsr_files = [
    'TSR_2024_코스피_처치추가_After포함.xlsx',
    'TSR_2024_코스닥_처치추가_After포함.xlsx',
    'TSR_2023_코스피_처치추가_After포함.xlsx',
    'TSR_2023_코스닥_처치추가_After포함.xlsx'
]
manufacture_file = '제조업통합.xlsx'

# ----------------------- 1. TSR + 재무 데이터 결합 -----------------------
tsr = pd.concat([pd.read_excel(os.path.join(BASE, f)) for f in tsr_files],
                ignore_index=True)

tsr['회사명'] = tsr['회사명'].astype(str).str.strip()

# 회계년도 숫자화 ---------------------------------------------------------
def to_int_year(x):
    try:
        return int(str(int(float(x)))[:4])
    except Exception:
        return None

tsr['연도'] = tsr['회계년도'].apply(to_int_year)

# ----------------------- 2. 제조업 기업명 필터링 -----------------------
mf_name = (pd.read_excel(os.path.join(BASE, manufacture_file))['회사명']
             .astype(str).str.strip().unique())
tsr = tsr[tsr['회사명'].isin(mf_name)].copy()

# ----------------------- 3. 기초 통계량 산출 -----------------------
cols_interest = [
    '매출액(영업수익)(*)(연결)(천원)',
    '[공통]부채(*)(IFRS)(천원)',
    '업력',
    '[공통]계속영업이익(손실)(3개월)(IFRS)(천원)',
    '자본금(*)(IFRS)(천원)',
    '[공통]자산(*)(IFRS)(천원)',
    '[공통]종업원'
]

# 숫자형 변환 & inf 처리
tsr[cols_interest] = (tsr[cols_interest]
                      .apply(pd.to_numeric, errors='coerce')
                      .replace([np.inf, -np.inf], np.nan))

# 연도별 통계 함수
def descriptive_table(df, year):
    tmp = df[df['연도'] == year][cols_interest]
    stats = tmp.agg(['count', 'mean', 'var', 'std', 'min', 'max']).T
    stats = stats.rename(columns={
        'count': 'N',
        'mean': '평균',
        'var': '분산',
        'std': '표준편차',
        'min': '최소',
        'max': '최대'
    })
    return stats

stats_2023 = descriptive_table(tsr, 2023)
stats_2024 = descriptive_table(tsr, 2024)

# ----------------------- 4. 결과 확인 -----------------------
print("\n=== 2023년 제조업 기초 통계량 ===")
print(stats_2023)

print("\n=== 2024년 제조업 기초 통계량 ===")
print(stats_2024)

# CSV 저장
stats_2023.to_csv(os.path.join(BASE, 'Descriptive_2023.csv'), encoding='utf-8-sig')
stats_2024.to_csv(os.path.join(BASE, 'Descriptive_2024.csv'), encoding='utf-8-sig')





=== 2023년 제조업 기초 통계량 ===
                                    N            평균            분산  \
매출액(영업수익)(*)(연결)(천원)           1237.0  1.540636e+09  1.121165e+20   
[공통]부채(*)(IFRS)(천원)            1574.0  3.823603e+08  6.517455e+18   
업력                             1575.0  2.977587e+01  3.297013e+02   
[공통]계속영업이익(손실)(3개월)(IFRS)(천원)  1484.0  4.917217e+05  4.864786e+15   
자본금(*)(IFRS)(천원)               1574.0  3.711422e+07  3.006674e+16   
[공통]자산(*)(IFRS)(천원)            1574.0  1.033214e+09  7.562339e+19   
[공통]종업원                        1575.0  6.965479e+02  1.820077e+07   

                                       표준편차            최소            최대  
매출액(영업수익)(*)(연결)(천원)           1.058851e+10  7.496400e+04  2.589355e+11  
[공통]부채(*)(IFRS)(천원)            2.552931e+09  6.039060e+05  7.206952e+10  
업력                             1.815768e+01  0.000000e+00  1.260000e+02  
[공통]계속영업이익(손실)(3개월)(IFRS)(천원)  6.974802e+07 -1.093256e+09  1.237560e+09  
자본금(*)(IFRS)(천원)               1.733976e+08  3.3910